## Feature Selection
* Only keep the important features

In [10]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from skimage import io

In [11]:
mypath = 'C:/Users/Yinzi/Desktop/ML PROJECT/Part 3/dataset/crop_part1'
filenames = np.array([f for f in listdir(mypath) if isfile(join(mypath, f))])
splitcolumns = [x.split('_')[0:3] + [mypath +'/' + x] for x in filenames if x.count('_') == 3]
filecolumns = ['age','gender','race','file']
df = pd.DataFrame(data = splitcolumns, columns = filecolumns)

In [12]:
images = []
for img_path in df['file']:
    img = io.imread(img_path)
    images.append(img)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, df['gender'], test_size=0.3, random_state=42)

In [16]:
from skimage import color
X_grays = []
for img in X_train:
    img_gray = color.rgb2gray(img)
    X_grays.append(img_gray)

In [17]:
X_flat = np.array(X_grays).reshape((6844, 200*200))
X_flat.shape

(6844, 40000)

## 1. Random Forest feature selection

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
X_flat_df = pd.DataFrame(X_flat)

In [38]:
X_flat_df.columns

RangeIndex(start=0, stop=40000, step=1)

In [27]:
# SelectFromModel will select those features which importance is greater than the mean importance of all the features by default
selector = SelectFromModel(RandomForestClassifier(n_estimators = 100))
selector.fit(X_flat_df, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [28]:
# To see which features are important
selector.get_support()

array([ True, False, False, ..., False,  True, False])

In [30]:
selected_feature= X_flat_df.columns[(selector.get_support())]
len(selected_feature)

12791

In [32]:
print(selected_feature[0:20])

Int64Index([0, 5, 7, 8, 9, 11, 12, 18, 19, 22, 23, 24, 28, 30, 31, 32, 34, 38,
            39, 42],
           dtype='int64')


In [45]:
importances = selector.estimator_.feature_importances_
importances[0:10]

array([5.78871241e-05, 1.12362207e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.30653797e-05, 2.02474167e-05, 5.43121539e-05,
       3.32301754e-05, 3.02619149e-05])

In [48]:
selected_df = X_flat_df.loc[:, selected_feature]
selected_df.shape

(6844, 12791)

In [49]:
X_selected_df = selected_df